In [1]:
import pandas as pd
import numpy as np
from pathlib2 import Path

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [3]:
def write_to_submission_file(predicted_labels, out_file, target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels, index = np.arange(1, predicted_labels.shape[0] + 1), columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [74]:
PATH_TO_DATA = Path('../../data/alice/')

times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv(PATH_TO_DATA / 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv(PATH_TO_DATA / 'test_sessions.csv',
                      index_col='session_id', parse_dates=times)

train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [75]:
train_df = train_df[train_df['time1'] > '2013-05-01 00:00']

In [50]:
#train_df = train_df.sort_values(by='time1')
#test_df = test_df.sort_values(by='time1')

In [76]:
class DataPreparator(BaseEstimator, TransformerMixin):
    """
    Fill NaN with zero values.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        return X[sites].fillna(0).astype('int')

In [77]:
class ListPreparator(BaseEstimator, TransformerMixin):
    """
    Prepare a CountVectorizer friendly 2D-list from data.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.values.tolist()
        # Convert dataframe rows to strings
        return [" ".join([str(site) for site in row]) for row in X]

In [78]:
class AttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new attributes to training and test set.
    """
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        # intraday features
        hour = X['time1'].apply(lambda ts: ts.hour)
        morning = ((hour >= 7) & (hour <= 11)).astype('int')
        day = ((hour >= 12) & (hour <= 18)).astype('int')
        evening = ((hour >= 19) & (hour <= 23)).astype('int')
        
        # season features
        month = X['time1'].apply(lambda ts: ts.month)
        summer = ((month >= 6) & (month <= 8)).astype('int')
        
        #winter = ((month == 12) | ((month <= 2) & (month >= 1))).astype('int')
        #spring = ((month >= 3) & (month <= 5)).astype('int')
        #autumn = ((month >= 9) & (month <= 11)).astype('int')
        
        # day of the week features
        weekday = X['time1'].apply(lambda ts: ts.weekday()).astype('int')
        
        # year features
        year = X['time1'].apply(lambda ts: ts.year).astype('int')
        #year_month = X['time1'].apply(lambda t: 100 * t.year + t.month).astype('int')
        
        #winter.values, spring.values, autumn.values,
        X = np.c_[morning.values, day.values, evening.values, summer.values, \
                  weekday.values, year.values]
        return X

In [79]:
class ScaledAttributesAdder(BaseEstimator, TransformerMixin):
    """
    Add new features, that should be scaled.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # session time features
        times = ['time%s' % i for i in range(1, 11)]
        # session duration: take to the power of 1/5 to normalize the distribution
        session_duration = (X[times].max(axis=1) - X[times].min(axis=1)).astype('timedelta64[ms]').astype(int) ** 0.2
        # number of sites visited in a session
        number_of_sites = X[times].isnull().sum(axis=1).apply(lambda x: 10 - x)
        # average time spent on one site during a session
        time_per_site = (session_duration / number_of_sites) ** 0.2
        
        X = np.c_[session_duration.values]
        return X

In [106]:
vectorizer_pipeline = Pipeline([
    ("preparator", DataPreparator()),
    ("list_preparator", ListPreparator()),
    ("vectorizer", TfidfVectorizer(ngram_range=(1, 3), max_features=50000))
])

attributes_pipeline = Pipeline([
    ("adder", AttributesAdder())
])

scaled_attributes_pipeline = Pipeline([
    ("adder", ScaledAttributesAdder()),
    ("scaler", StandardScaler())
])

In [107]:
full_pipeline = FeatureUnion(transformer_list=[
('vectorizer_pipeline', vectorizer_pipeline),
('attributes_pipeline', attributes_pipeline),
('scaled_attributes_pipeline', scaled_attributes_pipeline)
])

In [108]:
%%time

X_train = full_pipeline.fit_transform(train_df)
X_test = full_pipeline.transform(test_df)

y_train = train_df["target"].astype('int').values

CPU times: user 21.1 s, sys: 128 ms, total: 21.2 s
Wall time: 21.2 s


In [109]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [105]:
%%time
time_split = TimeSeriesSplit(n_splits=8)


logit = LogisticRegression(C=0.18, random_state=17, solver='liblinear')
#logit = LogisticRegression(random_state=17, solver='lbfgs')

cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                        scoring='roc_auc', n_jobs=1)

cv_scores.mean()

CPU times: user 30.3 s, sys: 14.5 s, total: 44.9 s
Wall time: 12.7 s


0.9325890417887861

In [98]:
logit.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]

write_to_submission_file(logit_test_pred, 'custom_pipeline_try_tfidf_4.csv')

In [110]:
%time
time_split = TimeSeriesSplit(n_splits=8)

c_val = np.logspace(-4, 2, 20)
logit_cv = LogisticRegressionCV(Cs=c_val, cv=time_split, \
                                scoring='roc_auc', solver='liblinear', \
                                n_jobs=-1, random_state=17)\
.fit(X_train, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [111]:
logit_cv.predict(X_train)
logit_cv.score(X_train, y_train)

/home/anton/jupyter_projects/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.9263621917607672

In [112]:
logit_test_pred = logit_cv.predict_proba(X_test)[:, 1]

write_to_submission_file(logit_test_pred, 'pipeline_cut_logitcv_tfidf.csv')

In [113]:
logit_cv

LogisticRegressionCV(Cs=array([1.00000000e-04, 2.06913808e-04, 4.28133240e-04, 8.85866790e-04,
       1.83298071e-03, 3.79269019e-03, 7.84759970e-03, 1.62377674e-02,
       3.35981829e-02, 6.95192796e-02, 1.43844989e-01, 2.97635144e-01,
       6.15848211e-01, 1.27427499e+00, 2.63665090e+00, 5.45559478e+00,
       1.12883789e+01, 2.33572147e+01, 4.83293024e+01, 1.00000000e+02]),
                     class_weight=None,
                     cv=TimeSeriesSplit(max_train_size=None, n_splits=8),
                     dual=False, fit_intercept=True, intercept_scaling=1.0,
                     l1_ratios=None, max_iter=100, multi_class='warn',
                     n_jobs=-1, penalty='l2', random_state=17, refit=True,
                     scoring='roc_auc', solver='liblinear', tol=0.0001,
                     verbose=0)